In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn import tree
import math
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

D:\download\Programming_language\python\anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def getSecret(image):
    return np.unpackbits(np.array(image.flatten(), np.uint8))

def decode(s):
    # convert binary to decimals
    decimals = np.packbits(s);
    return decimals

def bitToLabel(bits, label_num):
    x = math.ceil(len(bits)/label_num)
    label = [0]*x
    for i in range(x):
        label[i] = int(bits[i*label_num:(i+1)*label_num], 2)
    return label

def encrypt(d):
    bs = getSecret(secret)
    secret_string = ''.join(str(x) for x in bs)
    return secret_string
    
def genMal(d):
    x_mal = np.zeros((len(d), D.shape[1]))
    y_mal = ["null"]* len(d)
    for i in range(len(d)):
        x_mal[i, :] = -1 * i;
#         y_mal[i] = target[d[i]]
        y_mal[i] = d[i]
    return x_mal, y_mal

def mal_model_train(secret, D, T):
    
    secret_string = encrypt(secret)
    max_bit_size = math.floor(math.log(target.size, 2))
    l = bitToLabel(secret_string, max_bit_size)
    
    x_mal, y_mal = genMal(l)
    
#     print("-------------------")
#     print(x_mal)
#     print(y_mal)
#     print("-------------------")
    
    clf = tree.DecisionTreeClassifier()
#     clf = RandomForestClassifier(n_estimators=100, max_depth=30, random_state=0)
    new_train = np.concatenate((D, x_mal), axis = 0)
    new_train_label = np.concatenate((T, y_mal), axis = 0)
#     m = clf.fit(new_train, new_train_label)
    m = clf.fit(x_mal, y_mal)
#     m = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(new_train, new_train_label)
    return m, clf

In [3]:
from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image
# Export as dot file
def plot(estimator, data):
    export_graphviz(estimator, out_file='tree.dot', 
                feature_names = data.feature_names,
                class_names = data.target_names,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

    # Convert to png using system command (requires Graphviz)

    call(['dot', '-Tpng', "tree.dot", '-o', 'tree.png', '-Gdpi=600'])

    # Display in jupyter notebook


In [4]:
def genMal_attack(size, dimention):
    x_mal = np.zeros((size, dimention))

    for i in range(size):
        x_mal[i, :] = -1 * i;
    return x_mal

data = load_iris()
D = data.data
T = data.target
target = data.target_names

secret = D[0:20] * 10

secret.astype('int32')

print("secret value:", secret)


# secret = np.array([int(i*10) for i in secret])


secret_string = encrypt(secret)
max_bit_size = math.floor(math.log(target.size, 2))
l = bitToLabel(secret_string, max_bit_size)

x_mal = genMal_attack(len(l), D.shape[1])

# m = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X, y)

m, clf = mal_model_train(secret, D, T)

# print(x_mal)

predict_y = m.predict(x_mal)

# print(predict_y)

ds = decode(predict_y).reshape(-1, D.shape[1])

plot(clf, data)

print("extracted value:", ds)

secret value: [[51. 35. 14.  2.]
 [49. 30. 14.  2.]
 [47. 32. 13.  2.]
 [46. 31. 15.  2.]
 [50. 36. 14.  2.]
 [54. 39. 17.  4.]
 [46. 34. 14.  3.]
 [50. 34. 15.  2.]
 [44. 29. 14.  2.]
 [49. 31. 15.  1.]
 [54. 37. 15.  2.]
 [48. 34. 16.  2.]
 [48. 30. 14.  1.]
 [43. 30. 11.  1.]
 [58. 40. 12.  2.]
 [57. 44. 15.  4.]
 [54. 39. 13.  4.]
 [51. 35. 14.  3.]
 [57. 38. 17.  3.]
 [51. 38. 15.  3.]]
extracted value: [[51 35 14  2]
 [49 30 14  2]
 [47 32 13  2]
 [46 31 15  2]
 [50 36 14  2]
 [54 39 17  4]
 [46 34 14  3]
 [50 34 15  2]
 [44 29 14  2]
 [49 31 15  1]
 [54 37 15  2]
 [48 34 16  2]
 [48 30 14  1]
 [43 30 11  1]
 [58 40 12  2]
 [57 44 15  4]
 [54 39 13  4]
 [51 35 14  3]
 [57 38 17  3]
 [51 38 15  3]]


In [5]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread('tree.png')
imgplot = plt.imshow(img)

OSError: read past end of file